<a href="https://colab.research.google.com/github/mmmunyo/lec/blob/main/(%EC%B5%9C%EC%A2%85).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

excel_file_path = '/content/ilchim 240128 차트용 - 시험용.xlsx'  # Update to your actual file path
xls = pd.ExcelFile(excel_file_path)

sheet_data = []
row_names_list = []  # 각 시트의 행 이름을 저장
z_axis_names = xls.sheet_names  # Z축은 시트 이름들이 됩니다

for sheet_name in z_axis_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)

    # 첫 번째 열에서 행 이름을 추출하고 저장
    row_names = df.iloc[0:50, 0].tolist()
    row_names_list.append(row_names)

    # 데이터 처리를 시작하기 전에 NaN 값을 0으로 채움
    df = df.fillna(0)

    # 첫 번째 열(행 이름)을 제외하고 데이터 처리
    df_data = df.iloc[0:50, 1:]  # 첫 번째 행(열 이름)을 유지, 첫 번째 열을 제외
    data_array = df_data.to_numpy(dtype=float)  # 데이터를 float 타입의 numpy 배열로 변환

    # 처리된 데이터 배열을 추가
    sheet_data.append(data_array)

# 처리된 시트 데이터를 스택하여 3D 배열 생성
array_3d = np.stack(sheet_data, axis=0)

print(f"Shape of the 3D numpy array: {array_3d.shape}")

# 첫 번째 시트의 행 이름을 예시로 표시
print("Row Names from the First Sheet:", row_names_list[0])

# "ground floor" 데이터를 표시하여 두 번째 행이 포함되었는지 확인
print("Ground Floor (First Sheet) Data:")
for row_name, row_data in zip(row_names_list[0], array_3d[0, :5, :]):  # 첫 5행의 데이터를 예시로 표시
    print(f"{row_name}: {row_data}")

Shape of the 3D numpy array: (11, 50, 12)
Row Names from the First Sheet: ['口乾', '口渴', '口苦', '口臭', '口內炎.舌炎', '脣乾', '衄血', '夜啼', '上熱感', '惡寒', '惡風', '胸煩', '胸痛', '脇痛', '心悸', '短氣', '易怒', '太息', '易驚', '善恐', '不安', '憂鬱', '頭眩', '眼昏', '眼痒', '眼淚', '眼乾燥', '眼痛', '眼充血', '耳鳴', '耳痒', '耳聾', '無汗', '自汗', '盜汗', '疲勞', '睡眠', '腹脹滿', '小便頻數', '浮腫', '便溏', '泄瀉', '腰痛', '月經不順', '冷.崩漏.陰痒', '頭痛', '膝以下痛', '膝痛', '臀尖痛', '股關節痛']
Ground Floor (First Sheet) Data:
口乾: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
口渴: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
口苦: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
口臭: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
口內炎.舌炎: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [2]:
!pip install gradio

In [3]:
!pip install --upgrade typing-extensions
!pip install --upgrade pydantic
!pip check

import gradio as gr

ipython 7.34.0 requires jedi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
pygobject 3.42.1 requires pycairo, which is not installed.


In [4]:
# Load data from Excel
excel_file_path = '/content/ilchim 240128 차트용 - 시험용.xlsx'  # Update to your actual file path
xls = pd.ExcelFile(excel_file_path)

sheet_data = []
row_names_list = []  # Store row names for each sheet
z_axis_names = xls.sheet_names  # Z axis becomes sheet names
column_names = []  # Initialize list to store actual column names from the first sheet

# Process each sheet
for sheet_name in z_axis_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    row_names = df.iloc[:, 0].tolist()
    row_names_list.append(row_names)

    # Assuming the first row of the first sheet contains column names
    if not column_names:
        column_names = df.columns[1:].tolist()  # Skip the first column for row names

    df.fillna(0, inplace=True)
    data_array = df.iloc[:, 1:].to_numpy(dtype=float)
    sheet_data.append(data_array)

array_3d = np.stack(sheet_data, axis=0)

def calculate_top_values(selected_row_names):
    selected_indices = [row_names_list[0].index(name) for name in selected_row_names]
    selection_mask = np.zeros(array_3d.shape[1], dtype=bool)
    selection_mask[selected_indices] = True

    modified_data = np.zeros_like(array_3d)
    for layer_index in range(len(array_3d)):
        modified_data[layer_index] = array_3d[layer_index] * selection_mask[:, np.newaxis]

    results = []
    for layer_index, layer_data in enumerate(modified_data):
        for col_index, sum_val in enumerate(layer_data.sum(axis=0)):
            identifier = f"{column_names[col_index]} - {z_axis_names[layer_index]}"
            results.append((sum_val, identifier))

    top_results = sorted(results, key=lambda x: x[0], reverse=True)[:10]
    output = "\n".join([f"{name}: {value:.2f}" for value, name in top_results])
    return output

iface = gr.Interface(
    fn=calculate_top_values,
    inputs=gr.CheckboxGroup(choices=row_names_list[0], label="Select Rows"),
    outputs="text",
    title="Data Analysis Based on Row Selection",
    description="Select rows for analysis. Outputs the top 10 highest sums with actual column-layer names."
)

iface.launch()



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9fba8c2c6c9167013a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
